In [1]:
import cv2
import csv
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)

from tifffile import imread, imsave

Populating the interactive namespace from numpy and matplotlib


In [2]:
image_path = '/Users/prakash/Desktop/BobSegDataAndResults/GoodData/2018-06-05_fromMark/20180604_LP823_Control-03_Coupled/MyosinStack.tif'
src_img = imread(image_path)
img_size = src_img.shape
row_size = range(0,img_size[1])
col_size = range(0,img_size[2])
print(img_size)

(61, 896, 1086)


In [3]:
Xpts = range(0, img_size[1])
Ypts = range(0, img_size[2])
Points = np.array(list(itertools.product(Xpts, Ypts))).astype(float32)

In [4]:
image_index = 2 # The image time point(exclusive of) until which TSP is to be applied based on selected control points
flow_x = np.zeros(shape=(image_index,img_size[1],img_size[2]))
flow_y = np.zeros(shape=(image_index,img_size[1],img_size[2]))

In [5]:
for index in range(1, image_index,1):
    with open("/Users/prakash/Desktop/TPS/MarkData/time"+str(index)+"-"+str(index+1)+"/time_00"+str(index+1)+".csv","r") as source:
        rdr= csv.reader( source )
        with open("/Users/prakash/Desktop/TPS/MarkData/time"+str(index)+"-"+str(index+1)+"/"+"Resultstime"+str(index)+"-"+str(index+1)+".csv","w") as result:
            #with open("/Users/prakash/Desktop/BobSegDataAndResults/FlowQuantification/time"+str(i)+"-"+str(i+1)+"/"+"Resultstime"+str(i)+"-"+str(i+1)+".csv","w") as result:
            wtr= csv.writer( result )
            for r in rdr:
                wtr.writerow( (r[1], r[2]) )

    tps = cv2.createThinPlateSplineShapeTransformer()
    filename = "/Users/prakash/Desktop/TPS/MarkData/time"+str(index)+"-"+str(index+1)+"/"+"Resultstime"+str(index)+"-"+str(index+1)+".csv"
    input_file = open(filename,"r+")
    reader_file = csv.reader(input_file)
    value = len(list(reader_file))-1
    skip = int(value/2)
    sshape = np.genfromtxt(filename,delimiter=',', skip_header =1, max_rows=skip).astype(float32)
    tshape = np.genfromtxt(filename,delimiter=',', skip_header =skip+1, max_rows=skip).astype(float32)
    print(sshape)
    print(tshape)

    sshape = sshape.reshape(1,-1,2)
    tshape = tshape.reshape(1,-1,2)
    matches = list()

    for i in range(0, sshape.shape[1],1):
        matches.append(cv2.DMatch(i,i,0))

    tps.estimateTransformation(sshape,tshape,matches)  
    ret, tshape_ = tps.applyTransformation (Points.reshape(1,-1,2))
    target_warp = tshape_.reshape(img_size[1],img_size[2],2)
    print(target_warp.shape)

    # Creating flow
    tsp_flow_x = np.zeros(shape=(img_size[1],img_size[2]))
    tsp_flow_y = np.zeros(shape=(img_size[1],img_size[2]))

    for row in range(img_size[1]):
        for col in range(img_size[2]):
            tsp_flow_x[row][col] = target_warp[row][col][0] - row
            tsp_flow_y[row][col] = target_warp[row][col][1] - col
    flow_x[index-1,:,:] = tsp_flow_x
    flow_y[index-1,:,:] = tsp_flow_y

[[221.57849 587.1167 ]
 [235.51009 623.6041 ]
 [718.47156 531.3903 ]
 [273.32437 642.843  ]
 [320.4264  570.53143]
 [236.8369  642.17957]
 [823.29016 536.0342 ]
 [881.0067  468.36652]]
[[232.85645 592.42395]
 [237.5003  618.96027]
 [720.4618  538.0244 ]
 [275.3146  639.52594]
 [319.76297 573.8485 ]
 [248.77826 647.4869 ]
 [819.97314 539.35126]
 [877.6897  461.73242]]
(896, 1086, 2)
